# Llama2 cpp

- ポイント
    - cmake をインストール
    - export CMAKE_ARGS="-DLLAMA_CUBLAS=ON"　を設定
- cf. [Llama.cpp で Llama 2 を試す](https://note.com/npaka/n/n0ad63134fbe2)
- cf. [llama-cpp-python 0.1.77](https://pypi.org/project/llama-cpp-python/)


In [1]:
import torch


torch.cuda.is_available()

True

In [2]:
import pathlib

model_file = "../data/llama-2-7b-32k-instruct.Q8_0.gguf"
pathlib.Path(model_file).exists()

True

In [3]:
from llama_cpp import Llama

n_gqa = 8 if "70b" in model_file else 1
llm = Llama(model_path=model_file, n_gqa=n_gqa, n_gpu_layers=34)

ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 2 CUDA devices:
  Device 0: NVIDIA GeForce GTX 1080 Ti, compute capability 6.1, VMM: yes
  Device 1: NVIDIA GeForce GTX 1080 Ti, compute capability 6.1, VMM: yes
llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from ../data/llama-2-7b-32k-instruct.Q8_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = togethercomputer_llama-2-7b-32k-instruct
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llam

In [4]:
# prompt = "富士山の高さは？"
prompt = "1から20までの間の数で、5で終わる数は？"
prompt_formatted = f"""[INST] <<SYS>>
あなたは誠実で優秀な日本人のアシスタントです。<</SYS>>

{prompt} [/INST]
"""

In [5]:
# 推論の実行
for jsn in llm(
    prompt_formatted.strip(),
    max_tokens=256,
    temperature=0.0001,
    stop=[
        "Instruction:",
        "Input:",
        "Response:",
    ],
    echo=True,
    stream=True,
    repeat_penalty=1.1,
):
    print(jsn["choices"][0]["text"], sep="", end="")
print("")



あなたは誠実で優秀な日本人のアシスタントです。<</SYS>>

1から20までの間の数で、5で終わる数は？ [/INST]

5つの数を挙げます。

1. 15
2. 16
3. 17
4. 18
5. 19

あなたは、誠実で優秀な日本人のアシスタントです。<</SYS>>

あなたは、誠実で優秀な日本人のアシスタントです。<</SYS>>

あなたは、誠実で優秀な日本人のアシスタントです。<</SYS>>

あなたは、誠実で優秀な日本人のアシスタントです。<</SYS>>

あなたは、誠



llama_print_timings:        load time =     487.52 ms
llama_print_timings:      sample time =      87.79 ms /   258 runs   (    0.34 ms per token,  2938.73 tokens per second)
llama_print_timings: prompt eval time =     487.31 ms /    70 tokens (    6.96 ms per token,   143.65 tokens per second)
llama_print_timings:        eval time =   11019.37 ms /   257 runs   (   42.88 ms per token,    23.32 tokens per second)
llama_print_timings:       total time =   12098.91 ms


# LangChain

## ストリーミング用のコールバックをセットアップ

In [7]:
from typing import Any
from langchain_core.callbacks.base import BaseCallbackHandler


def handler_print(token: str):
    print(token, sep="", end="")


class StreamingCallbackHandlerSimple(BaseCallbackHandler):
    def on_llm_new_token(self, token: str, **kwargs: Any) -> None:
        handler_print(token)

In [16]:
from app.llama2cpp.component.llama2cpp import LlamaCppCustom

n_gqa = 8 if "70b" in model_file else 1
llm = LlamaCppCustom(
    model_path=model_file,
    n_ctx=512,
    temperature=0,
    max_tokens=256,
    n_gqa=n_gqa,
    n_gpu_layers=34,
    verbose=True,
    streaming=True,
)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from ../data/llama-2-7b-32k-instruct.Q8_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = togethercomputer_llama-2-7b-32k-instruct
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                

In [17]:
text = "富士山の高さは？正確に"

prompt_formatted = f"""[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
you have to answer in Japanese.
<</SYS>>

{text} [/INST]

"""

In [18]:
from langchain_core.runnables.config import RunnableConfig


config = RunnableConfig(callbacks=[StreamingCallbackHandlerSimple()])

In [19]:
for tkn in llm.stream(input=prompt_formatted, stop=None, config=config):
    # NOTE: printing each token in callback handler
    pass
print("")

富士山の高さは、3776メートルです。

これは、富士山が日本国内で最も高い山として知られているためです。富士山は、静岡県の富士五湖地域にあり、日本海上保安庁（JCG）が管理しています。

富士山は、毎年多くの観光客が訪れることで知られています。山頂には、展望台や神社などがあり、日本三景の一つ「富士の湖」も見下ろせます。

富士山は、毎年数万人の登山者が訪れることで知られています。登山道は多くあり、最高級の登山道である「頂上」には、日本三景の一つ「富士の湖」も見下ろ



llama_print_timings:        load time =     100.18 ms
llama_print_timings:      sample time =      97.37 ms /   256 runs   (    0.38 ms per token,  2629.12 tokens per second)
llama_print_timings: prompt eval time =    2242.50 ms /   160 tokens (   14.02 ms per token,    71.35 tokens per second)
llama_print_timings:        eval time =   11384.93 ms /   256 runs   (   44.47 ms per token,    22.49 tokens per second)
llama_print_timings:       total time =   14278.53 ms


In [20]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.schema import SystemMessage

template_messages = [
    SystemMessage(content="You are a helpful assistant."),
    MessagesPlaceholder(variable_name="chat_history"),
    HumanMessagePromptTemplate.from_template("{text}"),
]
prompt = ChatPromptTemplate.from_messages(template_messages)

## using LLMChain

In [21]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_experimental.chat_models import Llama2Chat


memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
model = Llama2Chat(llm=llm)
chain = LLMChain(
    llm=model,
    prompt=prompt,
    memory=memory,
)

In [22]:
memory.clear()
for tkn in chain.stream(input=text, config=config):
    # NOTE: printing each token in callback handler
    pass
print("")

Llama.generate: prefix-match hit




富士山の高さは3776.24mです。

富士山の標高は、日本三名山の中で最も高い。

富士山は、静岡県の富士市にあり、東京都と隣接しています。

富士山は、古くから人々の崇拝を集めてきた山で、日本三名山の中でも特に人気が高い。

富士山は、夏季には観光客で賑わうことが多く、冬季には登山者が多く訪れます。

富士山の標高は、日本三名山の中で最も高いため、日本の山々の中でも特に美しい景色を見下ろすことができます。

富士山は、日本の文化や伝統に深く関ling



llama_print_timings:        load time =     100.18 ms
llama_print_timings:      sample time =      92.45 ms /   256 runs   (    0.36 ms per token,  2768.97 tokens per second)
llama_print_timings: prompt eval time =     527.39 ms /    40 tokens (   13.18 ms per token,    75.84 tokens per second)
llama_print_timings:        eval time =   10486.06 ms /   255 runs   (   41.12 ms per token,    24.32 tokens per second)
llama_print_timings:       total time =   11630.36 ms


In [23]:
# print(chain.run(text=text))